# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## Note: this is for use with 3d slices of data in Zimmer lab format 

This is for use with real WBFM data; for now I don't have annotations, and am just going to make sure the pipeline works for extracting volumes

## Create a new project

It is always good idea to keep the projects seperate if you want to use different networks to analze your data. You should use one project if you are tracking similar subjects/items even if in different environments. This function creates a new project with sub-directories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos (for lableing more data) to the project at any stage of the project. 

In [1]:
%env DLClight=True
import deeplabcut

env: DLClight=True
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [2]:
task='WormTest' # Enter the name of your experiment Task
experimenter='Charlie' # Enter the name of the experimenter
video=['/users/charles.fieseler/test_worm1_data/mCherry/']

path_config_file=deeplabcut.create_new_project(task,experimenter,video,copy_videos=False, videotype='.tiff') 

# NOTE: The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
# path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-22/config.yaml'

Created "/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/videos"
Created "/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/labeled-data"
Created "/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/training-datasets"
Created "/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/dlc-models"
3  videos from the directory /users/charles.fieseler/test_worm1_data/mCherry/ were added to the project.
Attempting to create a symbolic link of the video ...
Created the symlink of /users/charles.fieseler/test_worm1_data/mCherry/test_100frames.ome.tiff to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/videos/test_100frames.ome.tiff
Created the symlink of /users/charles.fieseler/test_worm1_data/mCherry/test_5frames.ome.tiff to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/video

In [3]:
print(path_config_file)

/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/config.yaml


## Save individual volumes from ome.tiff movie

In [6]:
#
# Basically this is a custom extract_frames() function
#

from pathlib import Path
from deeplabcut.utils import auxiliaryfunctions
import tifffile
import platform
import os

In [13]:
        
def extract_volumes_from_charlie_output(path_config_file, nz, which_vol=None, which_slice=None):
    """
    Takes a video filename, which is a large ome-tiff file, and saves a volume in the folder given by 'out_folder'
    
    if which_slice is not None:
        saves single slices (instead of full volumes)
    """
    
    if which_slice is None:
        saving_str = "volume"
    else:
        saving_str = "slice"
    
    config_file = Path(path_config_file).resolve()
    cfg = auxiliaryfunctions.read_config(config_file)
    print("Config file read successfully.")
    
    video_fname = [i for i in cfg['video_sets'].keys()][0] # Assume one video for now (will be giant, ~3GB)
    print(video_fname)
    
    # Get volume indices to save
    if which_vol is None:
        which_vol = [0]
    
    for i_vol in which_vol:
        print("Reading {} {}/{}".format(saving_str, i_vol, len(which_vol)-1))
        
        # Convert scalar volume label to the sequential frames
        # Note: this may change for future input videos!
        if which_slice is None:
            vol_indices = list(range(i_vol*nz, i_vol*nz + nz))
        else:
            vol_indices = list(range(i_vol*nz + which_slice, i_vol*nz + 1 + which_slice))
        print("Converted {} indices: {} to {} (not including last frame)".format(saving_str, vol_indices[0], vol_indices[-1]))

        # Read and make output name
        this_volume = tifffile.imread(video_fname, key=vol_indices)
        output_name = 'img{}.tif'.format(i_vol)

        # Save in output folder
        fname = Path(video_fname)
        output_path = os.path.join(Path(path_config_file).parents[0],'labeled-data',fname.stem)

        tifffile.imsave(os.path.join(str(output_path),output_name), this_volume)

        print('Saved {} to {}'.format(saving_str, os.path.join(output_path, output_name)))
    

In [14]:
extract_volumes_from_charlie_output(path_config_file, 39, which_vol=list(range(5)), which_slice=13)

Config file read successfully.
/users/charles.fieseler/test_worm1_data/mCherry/test_100frames.ome.tiff
Reading slice 0/4
Converted slice indices: 13 to 13 (not including last frame)
Saved slice to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/labeled-data/test_100frames.ome/img0.tif
Reading slice 1/4
Converted slice indices: 52 to 52 (not including last frame)
Saved slice to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/labeled-data/test_100frames.ome/img1.tif
Reading slice 2/4
Converted slice indices: 91 to 91 (not including last frame)
Saved slice to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/labeled-data/test_100frames.ome/img2.tif
Reading slice 3/4
Converted slice indices: 130 to 130 (not including last frame)
Saved slice to /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/labeled-data/test_100frames.ome/img3.tif
Reading

# Separately: Manually do BCPD to create the annotations!

For now, is completely separate

# Automatically update the config file

# TODO; use edit_config()

In [23]:
import csv
import h5py
import pandas as pd
import os

In [31]:
def csv_annotations2config_names(path_config_file, num_dims=2):
    """
    Automatically updates the config file with the proper number of neurons, and deletes any other default bodyparts.
    Only affects the "bodyparts" field
    """
    
    # Get number of neurons from annotations
    home = os.path.dirname(path_config_file)
    # TODO: hardcoded folder
    annotations_fname = os.path.join(home,'labeled-data', 'test_100frames.ome','CollectedData_Charlie.csv')
    df = pd.read_csv(annotations_fname)
    num_neurons = int(df.shape[1] / num_dims)
    print("Adding body part annotations for {} neurons".format(num_neurons))
    
    # Read in entire config file into a list
    config_rows = []
    with open(path_config_file) as config:
        c_reader = csv.reader(config)#, delimiter=' ')
        for row in c_reader:
            config_rows.append(row)
    
    ## Delete the current bodypart lines
    delete_these_rows = False
    config_rows_edit = config_rows.copy()
    for row in config_rows:
        if row == ['bodyparts:']:
            delete_these_rows = True # Start deleting next row
        elif row == ['start: 0']:
            delete_these_rows = False # Do not delete this row, or others
            break
        elif delete_these_rows == True:
            # Don't delete either of the two above, but only in between those rows
            config_rows_edit.remove(row)
    
    ## Add in the named neuron lines
    # Using "list slicing" https://www.geeksforgeeks.org/python-insert-list-in-another-list/
    new_names = [['- neuron{}'.format(i)] for i in range(num_neurons)]
    insert_index = config_rows_edit.index(['start: 0'])
    config_rows_edit[insert_index:insert_index] = new_names
    
    ## Write the file again
    if True:
        with open(path_config_file, 'w', newline='') as config:
            c_writer = csv.writer(config)
            for row in config_rows_edit:
                c_writer.writerow(row)
    
    print("Finished! Check the config.yaml file to make sure the bodyparts are properly written")
    

In [29]:
def wb_tracker2config_names(path_config_file):
    """
    Automatically updates the config file with the proper number of neurons, and deletes any other default bodyparts.
    Only affects the "bodyparts" field
    """
    
    # Get number of neurons from annotations
    home = os.path.dirname(path_config_file)
    # TODO: hardcoded folder
    annotations_fname = os.path.join(home,'labeled-data', 'my-test-annotations','CollectedData_Charlie.csv')
#     with h5py.File(annotations_fname, 'r') as ann:
#         k = list(ann['df_with_missing']['table'].keys())
#         print(k)
#         num_neurons = int(len(ann[k[0]])/3)
    df = pd.read_csv(annotations_fname)
    num_neurons = int(df.shape[1] / 3)
    print("Adding body part annotations for {} neurons".format(num_neurons))
#     error()
    
    # Read in entire config file into a list
    config_rows = []
    with open(path_config_file) as config:
        c_reader = csv.reader(config)#, delimiter=' ')
        for row in c_reader:
            config_rows.append(row)
    
    ## Delete the current bodypart lines
    delete_these_rows = False
    config_rows_edit = config_rows.copy()
    for row in config_rows:
        if row == ['bodyparts:']:
            delete_these_rows = True # Start deleting next row
        elif row == ['start: 0']:
            delete_these_rows = False # Do not delete this row, or others
            break
        elif delete_these_rows == True:
            # Don't delete either of the two above, but only in between those rows
            config_rows_edit.remove(row)
    
    ## Add in the named neuron lines
    # Using "list slicing" https://www.geeksforgeeks.org/python-insert-list-in-another-list/
    new_names = [['- neuron{}'.format(i)] for i in range(num_neurons)]
    insert_index = config_rows_edit.index(['start: 0'])
    config_rows_edit[insert_index:insert_index] = new_names
    
    ## Write the file again
    if True:
        with open(path_config_file, 'w', newline='') as config:
            c_writer = csv.writer(config)
            for row in config_rows_edit:
                c_writer.writerow(row)
    #for row in config_rows_edit:
        #c_writer.writerow(row)
     #   print(row[:])
    
    

In [32]:
csv_annotations2config_names(path_config_file)

Adding body part annotations for 14 neurons
Finished! Check the config.yaml file to make sure the bodyparts are properly written


## Check the labels

[OPTIONAL] Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [7]:
%env DLClight=True
import deeplabcut
# path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/config.yaml'


env: DLClight=True


In [33]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Charlie.
They are stored in the following folder: /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/labeled-data/test_100frames.ome_labeled.
Attention: /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/labeled-data/test_5frames.ome does not appear to have labeled data!
Attention: /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/labeled-data/test_100frames.ome_old does not appear to have labeled data!
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use relauch the labeling GUI to move them around, save, and re-plot!

## Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [ ]:
# IF THE KERNEL IS RESET
%env DLClight=True
import deeplabcut
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/config.yaml'
# path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-22/config.yaml'
#path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

In [11]:
deeplabcut.create_training_dataset(path_config_file, augmenter_type="imgaug")
#remember, there are several networks you can pick, the default is resnet-50!

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95, 1, (array([3, 1, 4, 2]), array([0])))]

In [202]:
deeplabcut.create_training_dataset?

## Start training:

This function trains the network for a specific shuffle of the training dataset. 

In [1]:
# IF THE KERNEL IS RESET
%env DLClight=True
import deeplabcut
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/config.yaml'
# deeplabcut.__file__

env: DLClight=True
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


'/users/charles.fieseler/.conda/envs/DLC-GPU/lib/python3.6/site-packages/deeplabcut/__init__.py'

In [12]:
deeplabcut.train_network(path_config_file, displayiters=1000, saveiters=1000)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13]],
 'all_joints_names': ['neuron0',
                      'neuron1',
                      'neuron2',
                      'neuron3',
                      'neuron4',
                      'neuron5',
                      'neuron6',
                      'neuron7',
                      'neuron8',
                      'neuron9',
                      'neuron10',
                      'neuron11',
                      'neuron12',
                      'neuron13'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-2/UnaugmentedDataSet_WormTestJul7/WormTest_Charlie95shuffle1.mat',
 'dataset_type': 'imgaug',
 

Starting with imgaug pose-dataset loader.
Batch Size is 1
Initializing ResNet
Loading ImageNet-pretrained resnet_50
INFO:tensorflow:Restoring parameters from /users/charles.fieseler/.conda/envs/DLC-GPU/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt
Display_iters overwritten as 1000
Save_iters overwritten as 1000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/dlc-models/iteration-2/WormTestJul7-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, '

iteration: 1000 loss: 0.0345 lr: 0.005
iteration: 2000 loss: 0.0308 lr: 0.005
iteration: 3000 loss: 0.0305 lr: 0.005
iteration: 4000 loss: 0.0305 lr: 0.005
iteration: 5000 loss: 0.0283 lr: 0.005
iteration: 6000 loss: 0.0242 lr: 0.005
iteration: 7000 loss: 0.0148 lr: 0.005
iteration: 8000 loss: 0.0098 lr: 0.005
iteration: 9000 loss: 0.0075 lr: 0.005
iteration: 10000 loss: 0.0065 lr: 0.005
iteration: 11000 loss: 0.0316 lr: 0.02
iteration: 12000 loss: 0.0296 lr: 0.02


KeyboardInterrupt: 

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [5]:
# IF THE KERNEL IS RESET
%env DLClight=True
import deeplabcut
path_config_file = '/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-30/config.yaml'

#path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

env: DLClight=True


In [13]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

Config:
{'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13]],
 'all_joints_names': ['neuron0',
                      'neuron1',
                      'neuron2',
                      'neuron3',
                      'neuron4',
                      'neuron5',
                      'neuron6',
                      'neuron7',
                      'neuron8',
                      'neuron9',
                      'neuron10',
                      'neuron11',
                      'neuron12',
                      'neuron13'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-2/UnaugmentedDataSet_WormTestJul7/WormTest_Charlie95shuffle1.mat',
 'dataset_type': 'imgaug',
 

/users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/evaluation-results/  already exists!
Running  DLC_resnet50_WormTestJul7shuffle1_10000  with # of trainingiterations: 10000
Initializing ResNet
INFO:tensorflow:Restoring parameters from /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-07-07/dlc-models/iteration-2/WormTestJul7-trainset95shuffle1/train/snapshot-10000


0it [00:00, ?it/s]

Analyzing data...


5it [00:00,  9.90it/s]
  0%|          | 0/5 [00:00<?, ?it/s]

Done and results stored for snapshot:  snapshot-10000
Results for 10000  training iterations: 95 1 train error: 2.26 pixels. Test error: 24.23  pixels.
With pcutoff of 0.6  train error: 2.27 pixels. Test error: 4.87 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
Plotting...


100%|██████████| 5/5 [00:01<00:00,  4.57it/s]

The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [3]:
videofile_path = ['videos/video3.avi','videos/video4.avi'] #Enter a folder OR a list of videos to analyze.

deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.avi')

Using snapshot-11000 for model /users/charles.fieseler/DeepLabCut-dev/DeepLabCut/examples/WormTest-Charlie-2020-06-12/dlc-models/iteration-0/WormTestJun12-trainset95shuffle1
Initializing ResNet
No video(s) were found. Please check your paths and/or 'video_type'.


'DLC_resnet50_WormTestJun12shuffle1_11000'

## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [3]:
deeplabcut.extract_outlier_frames?

In [ ]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [ ]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)